In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import numpy as np
import time
from sklearn.metrics.pairwise import cosine_similarity
import torch
import re
from pyarrow import csv
import gc
import os
from os import listdir
from os.path import isfile, join
pd.set_option('display.max_colwidth', None)

In [ ]:
df_cluster_classification = pd.read_csv('Cluster_Prediction.csv')
df_cluster_classification = df_cluster_classification.drop(columns = 'Unnamed: 0')
df_tweet_lang2 = pd.read_csv("Latest_Tweet_Fake_Spreader_Tweets.csv", lineterminator='\n', usecols = ['Tweet_ID','User_ID', 'Tweet_Text','Tweet_Lang','Tweet_Time', 'Retweet_Count','Favorite_Count' ], dtype=str)
df_tweet_lang3 = pd.read_csv("Latest_Tweet_Fake_Spreader_Tweets.csv", lineterminator='\n', dtype=str)


In [ ]:
df_cluster_classification.Tweet_ID = df_cluster_classification.Tweet_ID.astype('str')
df_class_data = pd.merge(df_tweet_lang2, df_cluster_classification[['Cluster_ID', 'Tweet_ID', 'Tweet_Link', 'Prediction2']])

In [ ]:
df_class_data['Total_Engagement'] = df_class_data.Retweet_Count + df_class_data.Favorite_Count

In [ ]:
df_acc = pd.read_csv('All_Fake_without_debunkers_Updated.csv')

In [ ]:
df_acc_verified = df_acc.loc[df_acc.UserVerified == True]

In [ ]:
df_verified = df_acc_verified.User_ID.tolist()

In [ ]:
df_class_data.Tweet_ID = df_class_data.Tweet_ID.astype('str')
df_class_data.User_ID = df_class_data.User_ID.astype('int')

In [ ]:
df_class_data['From_verified'] = df_class_data['User_ID'].isin(df_verified)

In [ ]:
df_class_data['Total_Engagement'] = df_class_data.Retweet_Count.astype('int') + df_class_data.Favorite_Count.astype('int')

In [ ]:
df_class_data = df_class_data.assign(From_verified=df_class_data['From_verified'].astype('int'))

In [ ]:
df_class_data.User_ID = df_class_data.User_ID.astype('str')

# Hindutva

In [ ]:
df_class_data.groupby('Cluster_ID').sum().sort_values(by = 'From_verified')

In [ ]:
df_class_data.loc[df_class_data.Tweet_Text.str.contains('hindutva')|
                  df_class_data.Tweet_Text.str.contains('Hindutva')].groupby('Cluster_ID').count().reset_index(drop = False)

In [ ]:
df_hindutva_top_verified_list = df_class_data.loc[df_class_data.Tweet_Text.str.contains('hindutva')|df_class_data.Tweet_Text.str.contains('Hindutva')].groupby('Cluster_ID').sum().sort_values(by = ['From_verified', 'Total_Engagement']).reset_index().iloc[-5:].Cluster_ID.tolist()

In [ ]:
df_hindutva_top_verified_list

In [ ]:
df_class_data.loc[df_class_data.Cluster_ID.isin(df_hindutva_top_verified_list)].groupby('Cluster_ID').count()

In [ ]:
df_class_data.loc[df_class_data.Cluster_ID.isin(df_hindutva_top_verified_list)].groupby('Cluster_ID').sum()

In [ ]:
df_class_data.loc[df_class_data.Cluster_ID == df_hindutva_top_verified_list[0]][['Cluster_ID', 'Tweet_Time', 'Tweet_Text']]

In [ ]:
df_class_data.groupby('Cluster_ID').count().reset_index(drop = False)

In [ ]:
df_class_data = df_class_data.sort_values(['Cluster_ID', 'Tweet_Time'])

In [ ]:
df_class_data['Tweet_Date'] = pd.to_datetime(df_class_data["Tweet_Time"]).dt.date

In [ ]:
diff = df_class_data['Tweet_Date'].loc[0] - df_class_data['Tweet_Date'].loc[1]

In [ ]:
diff.days

In [ ]:
df_cluster = df_class_data.groupby('Cluster_ID').count().reset_index(drop = False).Cluster_ID.tolist()

In [ ]:
df_class_data.loc[df_class_data.Cluster_ID == 0].reset_index(drop = True).iloc[0].Tweet_Date

In [ ]:
df_class_data.loc[df_class_data.Cluster_ID == 0].reset_index(drop = True).iloc[-1].Tweet_Time

In [ ]:
date_diff_list = []
for i in df_cluster:
    last = df_class_data.loc[df_class_data.Cluster_ID == i].reset_index(drop = True).iloc[-1].Tweet_Date
    first = df_class_data.loc[df_class_data.Cluster_ID == i].reset_index(drop = True).iloc[0].Tweet_Date
    diff = last - first
    diff_dt = diff.days
    date_diff_list.append(diff_dt)

In [ ]:
df_cluster

In [ ]:
len(date_diff_list)

In [ ]:
df_cluster_interval = pd.DataFrame(list(zip(df_cluster, date_diff_list)),
              columns=['Cluster_ID','Time_Interval'])

In [ ]:
df_class_data

In [ ]:
df_class_data2 = pd.merge(df_class_data, df_cluster_interval)

In [ ]:
df_cluster_interval.sort_values(by = 'Time_Interval')

In [ ]:
df_class_data2

In [ ]:
keywords = ['Jai Shri Ram', 'जय श्रीराम', 'hindutva', 'Hindutva', 'muslim mob', 'Muslim mob', 'Stone', 'stone',
           'Love Jihad', 'Love jihad', 'love Jihad', 'love jihad', 'jihad', 'Jihad', '#LoveJihad', 'Akhand Bharat',
            'bulldozer', 'Bulldozer']
df_class_data.loc[df_class_data.Tweet_Text.str.contains('hindutva')|df_class_data.Tweet_Text.str.contains('Hindutva')].groupby('Cluster_ID').sum().sort_values(by = ['From_verified', 'Total_Engagement'])

In [ ]:
df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('hindutva')|
                  df_class_data2.Tweet_Text.str.contains('Hindutva')].groupby('Time_Interval').count()

In [ ]:
df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('hindutva')|
                  df_class_data2.Tweet_Text.str.contains('Hindutva')|
                  df_class_data2.Tweet_Text.str.contains('हिंदुत्व')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']].iloc[-20:]

In [ ]:
print('Total_Cluster '+ str(len(df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('hindutva')|
                  df_class_data2.Tweet_Text.str.contains('Hindutva')|
                  df_class_data2.Tweet_Text.str.contains('हिंदुत्व')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']].groupby('Cluster_ID').count())))

print('Total Tweets '+ str(len(df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('hindutva')|
                  df_class_data2.Tweet_Text.str.contains('Hindutva')|
                  df_class_data2.Tweet_Text.str.contains('हिंदुत्व')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']])))

In [ ]:
df_hindutva = [93905,45935,121455,9731,1694]

In [ ]:
df_class_data2[df_class_data2.Cluster_ID.isin(df_hindutva)].groupby('Cluster_ID').count()

In [ ]:
df_class_data2[df_class_data2.Cluster_ID.isin(df_hindutva)].groupby('Cluster_ID').sum()

In [ ]:
df_class_data2[df_class_data2.Cluster_ID.isin([17558, 27346, 112438, 40732, 27346, 129331, 3768,23463, 89168, 88626, 71101,29962,71798, 37923, 109784, 56897, 154899])][['Tweet_ID', 'Tweet_Text', 'Tweet_Link', 'Cluster_ID']].sort_values('Cluster_ID').to_csv('V:\SBERT All Embedding\Manual_Fake.csv', index = False)

In [ ]:
df_class_data2

In [ ]:
df_class_data2.loc[df_class_data2.Cluster_ID == 29962]

# Stone Pelters

In [ ]:
df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Stone pelter') | 
                  df_class_data2.Tweet_Text.str.contains('stone pelter') | 
                  df_class_data2.Tweet_Text.str.contains('Stone Pelter') | 
                  df_class_data2.Tweet_Text.str.contains('Stone-pelted') | 
                  df_class_data2.Tweet_Text.str.contains('stone-pelted') | 
                  df_class_data2.Tweet_Text.str.contains('Stone Pelted') |
                 df_class_data2.Tweet_Text.str.contains('stone pelted') |
                 df_class_data2.Tweet_Text.str.contains('stone pelting') |
                 df_class_data2.Tweet_Text.str.contains('Stone pelting') |
                 df_class_data2.Tweet_Text.str.contains('Stone Pelting') |
                 df_class_data2.Tweet_Text.str.contains('pelting stone')|
                 df_class_data2.Tweet_Text.str.contains('pelting stones')|
                  df_class_data2.Tweet_Text.str.contains('पत्थर')|
                  df_class_data2.Tweet_Text.str.contains('पत्थरबाज़')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']].iloc[-60:-40]

In [ ]:
len(df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Stone pelter') | 
                  df_class_data2.Tweet_Text.str.contains('stone pelter') | 
                  df_class_data2.Tweet_Text.str.contains('Stone Pelter') | 
                  df_class_data2.Tweet_Text.str.contains('Stone-pelted') | 
                  df_class_data2.Tweet_Text.str.contains('stone-pelted') | 
                  df_class_data2.Tweet_Text.str.contains('Stone Pelted') |
                 df_class_data2.Tweet_Text.str.contains('stone pelted') |
                 df_class_data2.Tweet_Text.str.contains('stone pelting') |
                 df_class_data2.Tweet_Text.str.contains('Stone pelting') |
                 df_class_data2.Tweet_Text.str.contains('Stone Pelting') |
                 df_class_data2.Tweet_Text.str.contains('pelting stone')|
                 df_class_data2.Tweet_Text.str.contains('pelting stones')|
                  df_class_data2.Tweet_Text.str.contains('पत्थर')|
                  df_class_data2.Tweet_Text.str.contains('पत्थरबाज़')].groupby('Cluster_ID').count())

In [ ]:
df_stone_pelt = [143337
,23402
,71766
,111879
,49573]

In [ ]:
df_class_data2[df_class_data2.Cluster_ID.isin(df_stone_pelt)].groupby('Cluster_ID').count()

In [ ]:
df_class_data2[df_class_data2.Cluster_ID.isin(df_stone_pelt)].groupby('Cluster_ID').sum()

In [ ]:
df_class_data2

# Muslim Mob

In [ ]:
df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('muslim mob')|
                   df_class_data2.Tweet_Text.str.contains('Muslim mob')| 
                   df_class_data2.Tweet_Text.str.contains('Muslim Mob')|
                   df_class_data2.Tweet_Text.str.contains('मुस्लिम')
                  ].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']]

In [ ]:
tj_max = '#DelhiLiquor'
df_class_data2.loc[df_class_data2.Tweet_Text.str.contains(tj_max)]

In [ ]:
print('Total_Cluster '+ str(len(df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('muslim mob')|
                   df_class_data2.Tweet_Text.str.contains('Muslim mob')| 
                   df_class_data2.Tweet_Text.str.contains('Muslim Mob')|
                   df_class_data2.Tweet_Text.str.contains('मुस्लिम')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']].groupby('Cluster_ID').count())))

print('Total Tweets '+ str(len(df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('muslim mob')|
                   df_class_data2.Tweet_Text.str.contains('Muslim mob')| 
                   df_class_data2.Tweet_Text.str.contains('Muslim Mob')|
                   df_class_data2.Tweet_Text.str.contains('मुस्लिम')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']])))

In [ ]:
df_class_data2[df_class_data2.Cluster_ID.isin([95996])]

In [ ]:
df_muslim_mob = [3768,
77895,
51432,
151287,
71798]

In [ ]:
df_class_data2[df_class_data2.Cluster_ID.isin(df_muslim_mob)].groupby('Cluster_ID').count()

In [ ]:
df_class_data2[df_class_data2.Cluster_ID.isin(df_muslim_mob)].groupby('Cluster_ID').sum()

# Love Jihad

In [ ]:
df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Love Jihad')|
                  df_class_data2.Tweet_Text.str.contains('Love jihad') |
                  df_class_data2.Tweet_Text.str.contains('love Jihad')|
                 df_class_data2.Tweet_Text.str.contains('love jihad')|
                  df_class_data2.Tweet_Text.str.contains('#LoveJihad') |
                  df_class_data2.Tweet_Text.str.contains('#lovejihad')|
                 df_class_data2.Tweet_Text.str.contains('#Lovejihad')|
                  df_class_data2.Tweet_Text.str.contains('लव जिहाद')|
                  df_class_data2.Tweet_Text.str.contains('#लवजिहाद')|
                  df_class_data2.Tweet_Text.str.contains('#लव_जिहाद')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']].iloc[-20:]

In [ ]:
print('Total_Cluster '+ str(len(df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Love Jihad')|
                  df_class_data2.Tweet_Text.str.contains('Love jihad') |
                  df_class_data2.Tweet_Text.str.contains('love Jihad')|
                 df_class_data2.Tweet_Text.str.contains('love jihad')|
                  df_class_data2.Tweet_Text.str.contains('#LoveJihad') |
                  df_class_data2.Tweet_Text.str.contains('#lovejihad')|
                 df_class_data2.Tweet_Text.str.contains('#Lovejihad')|
                  df_class_data2.Tweet_Text.str.contains('लव जिहाद')|
                  df_class_data2.Tweet_Text.str.contains('#लवजिहाद')|
                  df_class_data2.Tweet_Text.str.contains('#लव_जिहाद')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']].groupby('Cluster_ID').count())))

print('Total Tweets '+ str(len(df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Love Jihad')|
                  df_class_data2.Tweet_Text.str.contains('Love jihad') |
                  df_class_data2.Tweet_Text.str.contains('love Jihad')|
                 df_class_data2.Tweet_Text.str.contains('love jihad')|
                  df_class_data2.Tweet_Text.str.contains('#LoveJihad') |
                  df_class_data2.Tweet_Text.str.contains('#lovejihad')|
                 df_class_data2.Tweet_Text.str.contains('#Lovejihad')|
                  df_class_data2.Tweet_Text.str.contains('लव जिहाद')|
                  df_class_data2.Tweet_Text.str.contains('#लवजिहाद')|
                  df_class_data2.Tweet_Text.str.contains('#लव_जिहाद')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']])))

In [ ]:
df_love_jihad = [34891
,128613
,46445
,77750
,29962]

In [ ]:
df_class_data2[df_class_data2.Cluster_ID.isin(df_love_jihad)].groupby('Cluster_ID').count()

In [ ]:
df_class_data2[df_class_data2.Cluster_ID.isin(df_love_jihad)].groupby('Cluster_ID').sum()

# CCDH keywords

In [ ]:
df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('islam')
                ].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']]

In [ ]:
df_class_data2

# Jihad

In [ ]:
df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Jihad')|
                  df_class_data2.Tweet_Text.str.contains('jihad')|
                 df_class_data2.Tweet_Text.str.contains('जिहाद')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']].iloc[-20:]

In [ ]:
print('Total_Cluster '+ str(len(df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Jihad')|
                  df_class_data2.Tweet_Text.str.contains('jihad')|
                 df_class_data2.Tweet_Text.str.contains('जिहाद')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']].groupby('Cluster_ID').count())))

print('Total Tweets '+ str(len(df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Jihad')|
                  df_class_data2.Tweet_Text.str.contains('jihad')|
                 df_class_data2.Tweet_Text.str.contains('जिहाद')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']])))

In [ ]:
df_jihad = [46651,
20536,
45540,
46611,
23463]

In [ ]:
df_class_data2[df_class_data2.Cluster_ID.isin(df_jihad)].groupby('Cluster_ID').count()

In [ ]:
df_class_data2[df_class_data2.Cluster_ID.isin(df_jihad)].groupby('Cluster_ID').sum()

# Akhand Bharat

In [ ]:
df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Akhand Bharat')|
                  df_class_data2.Tweet_Text.str.contains('akhand bharat')|
                  df_class_data2.Tweet_Text.str.contains('अखण्ड भारत')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']].iloc[-20:]

In [ ]:
print('Total_Cluster '+ str(len(df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Akhand Bharat')|
                  df_class_data2.Tweet_Text.str.contains('akhand bharat')|
                  df_class_data2.Tweet_Text.str.contains('अखण्ड भारत')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']].groupby('Cluster_ID').count())))

print('Total Tweets '+ str(len(df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Akhand Bharat')|
                  df_class_data2.Tweet_Text.str.contains('akhand bharat')|
                  df_class_data2.Tweet_Text.str.contains('अखण्ड भारत')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']])))

In [ ]:
df_bharat = [89168,
59957,
18950,
39766,
122549]

In [ ]:
df_class_data2[df_class_data2.Cluster_ID.isin(df_bharat)].groupby('Cluster_ID').count()

In [ ]:
df_class_data2[df_class_data2.Cluster_ID.isin(df_bharat)].groupby('Cluster_ID').sum()

# Bulldozer

In [ ]:
df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Bulldozer')|
                  df_class_data2.Tweet_Text.str.contains('bulldozer')|
                  df_class_data2.Tweet_Text.str.contains('बुलडोज़र')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']].iloc[-20:]

In [ ]:
print('Total_Cluster '+ str(len(df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Bulldozer')|
                  df_class_data2.Tweet_Text.str.contains('bulldozer')|
                  df_class_data2.Tweet_Text.str.contains('बुलडोज़र')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']].groupby('Cluster_ID').count())))

print('Total Tweets '+ str(len(df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Bulldozer')|
                  df_class_data2.Tweet_Text.str.contains('bulldozer')|
                  df_class_data2.Tweet_Text.str.contains('बुलडोज़र')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']])))

In [ ]:
df_bull = [74738,
51730,
59305,
13784,
11480]

In [ ]:
df_class_data2[df_class_data2.Cluster_ID.isin(df_bull)].groupby('Cluster_ID').count()

In [ ]:
df_class_data2[df_class_data2.Cluster_ID.isin(df_bull)].groupby('Cluster_ID').sum()

In [ ]:
df_class_data2.groupby('Cluster_ID').sum().sort_values(by = 'Total_Engagement')

In [ ]:
df_class_data2.groupby('Cluster_ID').sum().sort_values(by = 'From_verified')

In [ ]:
df_class_data2.loc[df_class_data2.Cluster_ID == 13104]

In [ ]:
df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Biden')|
                  df_class_data2.Tweet_Text.str.contains('biden')].sort_values(by = 'Time_Interval')

In [ ]:
df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('russia')|
                  df_class_data2.Tweet_Text.str.contains('Russia')].sort_values(by = 'Time_Interval')

In [ ]:
df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Adani')|
                  df_class_data2.Tweet_Text.str.contains('adani')].sort_values(by = 'Time_Interval')

In [ ]:
df_class_sum_scplot = df_class_data2.groupby('Cluster_ID').sum()

In [ ]:
df_class_sum_scplot = df_class_sum_scplot.reset_index(drop = False)

In [ ]:
df_class_count_scplot = df_class_data2.groupby('Cluster_ID').count().reset_index(drop = False)[['Cluster_ID', 'Tweet_ID']]

In [ ]:
df_non_verified = df_class_sum_scplot.loc[df_class_sum_scplot.From_verified == 0]

In [ ]:
df_verified = df_class_sum_scplot.loc[df_class_sum_scplot.From_verified > 0]

In [ ]:
df_non_verified = pd.merge(df_class_count_scplot, df_non_verified)
df_verified = pd.merge(df_class_count_scplot, df_verified)

In [ ]:
df_verified = df_verified.rename(columns = {'Tweet_ID':'Cluster_Size'})
df_non_verified = df_non_verified.rename(columns = {'Tweet_ID':'Cluster_Size'})

In [ ]:
df_non_verified.groupby('Cluster_ID').sum()

In [ ]:
import seaborn as sns
pd.set_option('display.max_colwidth', None)
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
from matplotlib.pyplot import figure

figure(figsize=(6, 4), dpi=80)

In [ ]:
df_verified['Verification'] = 'Verified'
df_non_verified['Verification'] = 'Non-Verified'

In [ ]:

ax = sns.scatterplot(data=df_verified, x="Cluster_Size", y="Total_Engagement") 




ax.set_xlabel("Cluster Size")

ax.set_ylabel("Total Engagement (Log)")

plt.show() 

In [ ]:

ax = sns.scatterplot(data=df_non_verified, x="Cluster_Size", y="Total_Engagement") 

ax.set_xlabel("Cluster Size")

ax.set_ylabel("Total Engagement (Log)")
plt.show() 

In [ ]:
frames1 = [df_verified, df_non_verified]
df_ver_non_ver = pd.concat(frames1)

In [ ]:
df_ver_non_ver = df_ver_non_ver.sort_values(by = 'Cluster_ID').reset_index(drop = True)

In [ ]:
df_ver_non_ver = df_ver_non_ver.drop(columns = 'Time_Interval')

In [ ]:
df_ver_non_ver = pd.merge(df_ver_non_ver, df_cluster_interval)

In [ ]:
plt.figure(figsize=(6,4)) 
ax = sns.scatterplot(data=df_ver_non_ver, x="Cluster_Size", y="Total_Engagement",
                hue="Verification",  palette=['#e31a1c','#1f78b4'], alpha = 0.7)

ax.set_xlabel("Cluster Size")
ax.set_ylabel("Total Engagement (Log)")
plt.yscale('log')
plt.show()

In [ ]:
plt.figure(figsize=(6,4)) 
ax = sns.scatterplot(data=df_ver_non_ver, x="Cluster_Size", y="Total_Engagement",
                hue="Verification",  palette=['#e31a1c','#1f78b4'], alpha = 0.7, style="Verification")

ax.set_xlabel("Cluster Size")
ax.set_ylabel("Total Engagement (Log)")
plt.yscale('log')
plt.xlim(-5, 200)
plt.show() 

In [ ]:
plt.figure(figsize=(6,4)) 
ax = sns.scatterplot(data=df_ver_non_ver, x="Time_Interval", y="Total_Engagement",
                hue="Verification",  palette=['#e31a1c','#1f78b4'], alpha = 0.7, style="Verification")

ax.set_xlabel("Time Interval")
ax.set_ylabel("Total Engagement (Log)")
plt.yscale('log')
plt.show() 

In [ ]:
plt.figure(figsize=(6,4)) 
ax = sns.scatterplot(data=df_ver_non_ver, x="Time_Interval", y="Total_Engagement",
                hue="Verification",  palette=['#e31a1c','#1f78b4'], alpha = 0.7, style="Verification")

ax.set_xlabel("Time Interval")

ax.set_ylabel("Total Engagement (Log)")
plt.show() 

In [ ]:
plt.figure(figsize=(4,3)) 
ax = sns.boxplot( x="Verification",y="Total_Engagement", data=df_ver_non_ver, showfliers=False, width=0.3)


ax.set_xlabel("Verification")

ax.set_ylabel("Total Engagement")
plt.show()

In [ ]:
df_ver_non_ver

In [ ]:
df_ver_non_ver

In [ ]:
df_ver_non_ver.loc[df_ver_non_ver.Verification == 'Verified'].Total_Engagement.describe()

In [ ]:
df_ver_non_ver.loc[df_ver_non_ver.Verification == 'Non-Verified'].Total_Engagement.describe()

In [ ]:
df_ver_non_ver.to_csv('Ver_NonVer.csv')

In [ ]:
plt.figure(figsize=(4,3))
ax = sns.boxplot( x="Verification",y="Cluster_Size", data=df_ver_non_ver, showfliers=False, width=0.3)


ax.set_xlabel("Verification")

ax.set_ylabel("Cluster Size")
plt.show()

In [ ]:
plt.figure(figsize=(4,3))
ax = sns.boxplot( x="Verification", y="Time_Interval", data=df_ver_non_ver, showfliers=False, width=0.3)


ax.set_xlabel("Verification")

ax.set_ylabel("Time Interval")
plt.show()

In [ ]:
df_ver_non_ver.sort_values(by = 'Time_Interval')

In [ ]:
df_class_data.loc[df_class_data.Cluster_ID == 0].sort_values(by = 'Tweet_Time').iloc[0].Tweet_Date - df_class_data.loc[df_class_data.Cluster_ID == 0].sort_values(by = 'Tweet_Time').iloc[-1].Tweet_Date

In [ ]:
df_ver_non_ver.loc[df_ver_non_ver.Cluster_ID == 0]

In [ ]:
df_non_verified.loc[df_non_verified.Total_Engagement > 50000]

In [ ]:
df_class_data.loc[df_class_data.Cluster_ID == 48755]

# Time Interval Check

In [ ]:
df_time_check = [143337,
23402,
71766,
111879,
49573]

In [ ]:
df_ver_non_ver[df_ver_non_ver.Cluster_ID.isin(df_time_check)]

# International Topic Analysis

# Russia-Ukraine

In [ ]:
df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Ukraine')|
                  df_class_data2.Tweet_Text.str.contains('ukraine')|
                  df_class_data2.Tweet_Text.str.contains('Russia')|
                  df_class_data2.Tweet_Text.str.contains('russia')|
                  df_class_data2.Tweet_Text.str.contains('रूस')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']].iloc[-110:-80]

In [ ]:
df_rus = [2092, 67278,2350,49367,55518,121487]

In [ ]:
print('Total_Cluster '+ str(len(df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Ukraine')|
                  df_class_data2.Tweet_Text.str.contains('ukraine')|
                  df_class_data2.Tweet_Text.str.contains('Russia')|
                  df_class_data2.Tweet_Text.str.contains('russia')|
                  df_class_data2.Tweet_Text.str.contains('रूस')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']].groupby('Cluster_ID').count())))

print('Total Tweets '+ str(len(df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Ukraine')|
                  df_class_data2.Tweet_Text.str.contains('ukraine')|
                  df_class_data2.Tweet_Text.str.contains('Russia')|
                  df_class_data2.Tweet_Text.str.contains('russia')|
                  df_class_data2.Tweet_Text.str.contains('रूस')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']])))

In [ ]:
df_class_data2[df_class_data2.Cluster_ID.isin(df_rus)].groupby('Cluster_ID').count()

In [ ]:
df_class_data2[df_class_data2.Cluster_ID.isin(df_rus)].groupby('Cluster_ID').sum()

In [ ]:
df_class_data2[df_class_data2.Cluster_ID.isin(df_rus)][['Cluster_ID', 'Time_Interval']]

# Pakistan Imran Khan Bangladesh

In [ ]:
df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Pakistan')|
                  df_class_data2.Tweet_Text.str.contains('pakistan')|
                  df_class_data2.Tweet_Text.str.contains('पाकिस्तान')
                  ].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']].iloc[-90:-60]

In [ ]:
print('Total_Cluster '+ str(len(df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Pakistan')|
                  df_class_data2.Tweet_Text.str.contains('pakistan')|
                  df_class_data2.Tweet_Text.str.contains('पाकिस्तान')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']].groupby('Cluster_ID').count())))

print('Total Tweets '+ str(len(df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Pakistan')|
                  df_class_data2.Tweet_Text.str.contains('pakistan')|
                  df_class_data2.Tweet_Text.str.contains('पाकिस्तान')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']])))

In [ ]:
df_pak = [1906,
39614,
3768,
14446,
129331]

In [ ]:
df_class_data2[df_class_data2.Cluster_ID.isin(df_pak)].groupby('Cluster_ID').count()

In [ ]:
df_class_data2[df_class_data2.Cluster_ID.isin(df_pak)].groupby('Cluster_ID').sum()

In [ ]:
df_class_data2[df_class_data2.Cluster_ID.isin(df_pak)][['Cluster_ID', 'Time_Interval']].drop_duplicates()

In [ ]:
df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Bangladesh')|
                  df_class_data2.Tweet_Text.str.contains('bangladesh')|
                  df_class_data2.Tweet_Text.str.contains('बांग्लादेश')
                  ].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']].iloc[-150:-120]

In [ ]:
df_bang = [66620,
98071,
49767,
16664,
3665]

In [ ]:
print('Total_Cluster '+ str(len(df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Bangladesh')|
                  df_class_data2.Tweet_Text.str.contains('bangladesh')|
                  df_class_data2.Tweet_Text.str.contains('बांग्लादेश')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']].groupby('Cluster_ID').count())))

print('Total Tweets '+ str(len(df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Bangladesh')|
                  df_class_data2.Tweet_Text.str.contains('bangladesh')|
                  df_class_data2.Tweet_Text.str.contains('बांग्लादेश')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']])))

In [ ]:
print(df_class_data2[df_class_data2.Cluster_ID.isin(df_bang)].groupby('Cluster_ID').count()[['Tweet_ID']])
print("\n")
print(df_class_data2[df_class_data2.Cluster_ID.isin(df_bang)].groupby('Cluster_ID').sum())
print(df_class_data2[df_class_data2.Cluster_ID.isin(df_bang)][['Cluster_ID', 'Time_Interval']].drop_duplicates())

# Trump

In [ ]:
df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Trump')|
                  df_class_data2.Tweet_Text.str.contains('Donlad Trump')|
                  df_class_data2.Tweet_Text.str.contains('ट्रम्प')
                  ].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']].iloc[-30:]

In [ ]:
df_trump = [146600,
151288,
7341,
41746,
14495]
df_q = df_trump

In [ ]:
print('Total_Cluster '+ str(len(df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Trump')|
                  df_class_data2.Tweet_Text.str.contains('Donlad Trump')|
                  df_class_data2.Tweet_Text.str.contains('ट्रम्प')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']].groupby('Cluster_ID').count())))

print('Total Tweets '+ str(len(df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Trump')|
                  df_class_data2.Tweet_Text.str.contains('Donlad Trump')|
                  df_class_data2.Tweet_Text.str.contains('ट्रम्प')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']])))

In [ ]:
print(df_class_data2[df_class_data2.Cluster_ID.isin(df_q)].groupby('Cluster_ID').count()[['Tweet_ID']])
print("\n")
print(df_class_data2[df_class_data2.Cluster_ID.isin(df_q)].groupby('Cluster_ID').sum())
print(df_class_data2[df_class_data2.Cluster_ID.isin(df_q)][['Cluster_ID', 'Time_Interval']].drop_duplicates())

In [ ]:
df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Putin')|
                  df_class_data2.Tweet_Text.str.contains('पुतिन')
                  ].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']].iloc[-30:]

In [ ]:
print('Total_Cluster '+ str(len(df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Putin')|
                  df_class_data2.Tweet_Text.str.contains('पुतिन')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']].groupby('Cluster_ID').count())))

print('Total Tweets '+ str(len(df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Putin')|
                  df_class_data2.Tweet_Text.str.contains('पुतिन')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']])))

In [ ]:
df_putin = [34196,
116672,
89885,
49216,
22727]
df_q = df_putin

In [ ]:
print(df_class_data2[df_class_data2.Cluster_ID.isin(df_q)].groupby('Cluster_ID').count()[['Tweet_ID']])
print("\n")
print(df_class_data2[df_class_data2.Cluster_ID.isin(df_q)].groupby('Cluster_ID').sum())
print(df_class_data2[df_class_data2.Cluster_ID.isin(df_q)][['Cluster_ID', 'Time_Interval']].drop_duplicates())

In [ ]:
df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Biden')|
                  df_class_data2.Tweet_Text.str.contains('बिडेन')
                  ].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']].iloc[-30:]

In [ ]:
print('Total_Cluster '+ str(len(df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Biden')|
                  df_class_data2.Tweet_Text.str.contains('बिडेन')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']].groupby('Cluster_ID').count())))

print('Total Tweets '+ str(len(df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Biden')|
                  df_class_data2.Tweet_Text.str.contains('बिडेन')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']])))

In [ ]:
df_biden =[95996,
89895,
40427,
154137,
134576]
df_q = df_biden

In [ ]:
print(df_class_data2[df_class_data2.Cluster_ID.isin(df_q)].groupby('Cluster_ID').count()[['Tweet_ID']])
print("\n")
print(df_class_data2[df_class_data2.Cluster_ID.isin(df_q)].groupby('Cluster_ID').sum())
print(df_class_data2[df_class_data2.Cluster_ID.isin(df_q)][['Cluster_ID', 'Time_Interval']].drop_duplicates())

# Kashmiri Files

In [ ]:
df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('The Kashmir Files')|
                  df_class_data2.Tweet_Text.str.contains('#TheKashmirFiles')
                  ].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']].iloc[-30:]

In [ ]:
df_kashmir =[45560,
28358,
16104,
32391,
4701]
df_q = df_kashmir

In [ ]:
print(df_class_data2[df_class_data2.Cluster_ID.isin(df_q)].groupby('Cluster_ID').count()[['Tweet_ID']])
print("\n")
print(df_class_data2[df_class_data2.Cluster_ID.isin(df_q)].groupby('Cluster_ID').sum())
print(df_class_data2[df_class_data2.Cluster_ID.isin(df_q)][['Cluster_ID', 'Time_Interval']].drop_duplicates())

In [ ]:
print('Total_Cluster '+ str(len(df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('The Kashmir Files')|
                  df_class_data2.Tweet_Text.str.contains('#TheKashmirFiles')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']].groupby('Cluster_ID').count())))

print('Total Tweets '+ str(len(df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('The Kashmir Files')|
                  df_class_data2.Tweet_Text.str.contains('#TheKashmirFiles')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']])))

# Boycott

In [ ]:
df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('boycott')|
                   df_class_data2.Tweet_Text.str.contains('Boycott')
                  ].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']].iloc[-30:]

In [ ]:
df_boy =[41685,
59794,
27564,
138993,
155385]
df_q = df_boy

In [ ]:
print(df_class_data2[df_class_data2.Cluster_ID.isin(df_q)].groupby('Cluster_ID').count()[['Tweet_ID']])
print("\n")
print(df_class_data2[df_class_data2.Cluster_ID.isin(df_q)].groupby('Cluster_ID').sum())
print(df_class_data2[df_class_data2.Cluster_ID.isin(df_q)][['Cluster_ID', 'Time_Interval']].drop_duplicates())

In [ ]:
print('Total_Cluster '+ str(len(df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('boycott')|
                   df_class_data2.Tweet_Text.str.contains('Boycott')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']].groupby('Cluster_ID').count())))

print('Total Tweets '+ str(len(df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('boycott')|
                   df_class_data2.Tweet_Text.str.contains('Boycott')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']])))

# Adani


In [ ]:
df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Adani')|
                   df_class_data2.Tweet_Text.str.contains('अदाणी')
                  ].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']].iloc[-30:]

In [ ]:
df_adani =[23271,
23270,
91812,
56279,
167878]
df_q = df_adani

In [ ]:
print(df_class_data2[df_class_data2.Cluster_ID.isin(df_q)].groupby('Cluster_ID').count()[['Tweet_ID']])
print("\n")
print(df_class_data2[df_class_data2.Cluster_ID.isin(df_q)].groupby('Cluster_ID').sum())
print(df_class_data2[df_class_data2.Cluster_ID.isin(df_q)][['Cluster_ID', 'Time_Interval']].drop_duplicates())

In [ ]:
print('Total_Cluster '+ str(len(df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Adani')|
                   df_class_data2.Tweet_Text.str.contains('अदाणी')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']].groupby('Cluster_ID').count())))

print('Total Tweets '+ str(len(df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Adani')|
                   df_class_data2.Tweet_Text.str.contains('अदाणी')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']])))

In [ ]:
df_class_data2.head()

In [ ]:
df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('muslim mob')|
                   df_class_data2.Tweet_Text.str.contains('Muslim mob')| 
                   df_class_data2.Tweet_Text.str.contains('Muslim Mob')|
                   df_class_data2.Tweet_Text.str.contains('मुस्लिम')
                  ].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']].iloc[-330:-300]

In [ ]:
df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('russia')|
                  df_class_data2.Tweet_Text.str.contains('Russia')].sort_values(by = 'Time_Interval')

In [ ]:
df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('covid')|
                  df_class_data2.Tweet_Text.str.contains('Covid')].sort_values(by = 'Time_Interval')

In [ ]:
df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('farmer')|
                  df_class_data2.Tweet_Text.str.contains('Farmer')].sort_values(by = 'Time_Interval')

In [ ]:
df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('pakistan')|
                  df_class_data2.Tweet_Text.str.contains('Pakistan')].sort_values(by = 'Time_Interval')

In [ ]:
df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('bangladesh')|
                  df_class_data2.Tweet_Text.str.contains('Bangladesh')].sort_values(by = 'Time_Interval')

In [ ]:
df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('cab')|
                  df_class_data2.Tweet_Text.str.contains('CAB')].sort_values(by = 'Time_Interval')

In [ ]:
df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('caa')|
                  df_class_data2.Tweet_Text.str.contains('CAA')].sort_values(by = 'Time_Interval')

In [ ]:
df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('#IndiaSupportsCAA')|
                  df_class_data2.Tweet_Text.str.contains('#IndiaSupportsCAB')].sort_values(by = 'Time_Interval')

In [ ]:
df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Trump')|
                  df_class_data2.Tweet_Text.str.contains('trump')].sort_values(by = 'Time_Interval')

In [ ]:
df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('#USABetrayedIndia')].sort_values(by = 'Time_Interval')

In [ ]:
df_class_data2.loc[df_class_data2.Tweet_Text.str.contains('Boycott')].sort_values(by = 'Time_Interval')[['Tweet_ID', 'Tweet_Text', 'Time_Interval', 'Cluster_ID']].iloc[-20:]

# Hashtag Analysis

In [ ]:
df_class_data2['HashTags'] = df_class_data2.Tweet_Text.str.findall(r'#.*?(?=\s|$)')

In [ ]:
df_hash_data = df_class_data2[['Tweet_ID', 'Cluster_ID','HashTags']]

In [ ]:
df_hash_only = df_hash_data.explode('HashTags').dropna()

In [ ]:
df_hash_only.drop_duplicates('HashTags')

In [ ]:
df_hash_only.groupby('HashTags').count().reset_index(drop = False).sort_values(by = 'Tweet_ID').iloc[-50:]

In [ ]:
df_hash_only

In [ ]:
df_hash_only.groupby('HashTags').count().reset_index(drop = False).sort_values(by = 'Tweet_ID').iloc[-50:]

# Spike Detection in Cluster Time Interval

In [ ]:
df_ver_non_ver.loc[(df_ver_non_ver.Time_Interval >= 30) & (df_ver_non_ver.Cluster_Size >= 10)].sample(5)

In [ ]:
df_class_data.loc[df_class_data.Cluster_ID == 35296]

In [ ]:
df_class_data2

In [ ]:
df_class_data2.Tweet_Date = pd.to_datetime(df_class_data2.Tweet_Time, errors='coerce')

In [ ]:
df_quest = df_class_data2.groupby('Cluster_ID').resample('M', on='Tweet_Date').count()

In [ ]:
df_quest = df_quest.rename(columns = {'Cluster_ID': 'Cluster_ID2', 'Tweet_Date':'Tweet2Date'}).reset_index()

In [ ]:
df_class_data2.to_csv('All_Tweet.csv')

In [ ]:
df_class_data2

In [ ]:
df_rater = pd.read_csv('Rater_Verification.csv')

In [ ]:
df_rater.Cluster_ID.tolist()

In [ ]:
df_mis_list = [17558, 27346, 112438, 40732, 27346, 129331, 3768,23463, 89168, 88626, 71101,29962,71798, 37923, 109784, 56897, 154899]

In [ ]:
df_class_data2[df_class_data2.Cluster_ID.isin(df_rater.Cluster_ID.tolist())].groupby('Cluster_ID').count()

In [ ]:
df_class_data2[df_class_data2.Cluster_ID.isin(df_mis_list)].groupby('Cluster_ID').count()